## Stabilization, P = 1.4

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_max = 1.4;
t_max = 5.0;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 6.0, tauP = 10.0, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_stabilization.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [3]:
hoomd.run(1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 756146 / 856000 | TPS 14.526 | ETA 01:54:34
Time 00:00:20 | Step 756293 / 856000 | TPS 14.5527 | ETA 01:54:11
Time 00:00:30 | Step 756453 / 856000 | TPS 15.9682 | ETA 01:43:54
Time 00:00:40 | Step 756615 / 856000 | TPS 16.1803 | ETA 01:42:22
Time 00:00:50 | Step 756785 / 856000 | TPS 16.8561 | ETA 01:38:05
Time 00:01:00 | Step 756961 / 856000 | TPS 17.4297 | ETA 01:34:42
Time 00:01:10 | Step 757138 / 856000 | TPS 17.6524 | ETA 01:33:20
Time 00:01:20 | Step 757304 / 856000 | TPS 16.5199 | ETA 01:39:34
Time 00:01:30 | Step 757473 / 856000 | TPS 16.8747 | ETA 01:37:18
Time 00:01:40 | Step 757652 / 856000 | TPS 17.7628 | ETA 01:32:16
Time 00:01:50 | Step 757840 / 856000 | TPS 18.6686 | ETA 01:27:38
Time 00:02:00 | Step 758015 / 85

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x116fee940>),
             ('particles', <hoomd.data.particle_data at 0x116fee048>),
             ('number_density', 0.7645431532464331),
             ('bonds', <hoomd.data.bond_data at 0x116fee0f0>),
             ('angles', <hoomd.data.angle_data at 0x116fee160>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x116fee1d0>),
             ('impropers', <hoomd.data.dihedral_data at 0x116fee208>),
             ('constraints', <hoomd.data.constraint_data at 0x116fee278>),
             ('pairs', <hoomd.data.bond_data at 0x116fee2e8>),
             ('timestep', 856000)])

In [ ]:
hoomd.run(1e5)

** starting run **
Time 03:09:06 | Step 856178 / 956000 | TPS 17.6648 | ETA 01:34:10
Time 03:09:16 | Step 856317 / 956000 | TPS 13.7505 | ETA 02:00:49
Time 03:09:26 | Step 856456 / 956000 | TPS 13.8675 | ETA 01:59:38
Time 03:09:36 | Step 856616 / 956000 | TPS 15.8828 | ETA 01:44:17
Time 03:09:46 | Step 856797 / 956000 | TPS 18.0069 | ETA 01:31:49
Time 03:09:56 | Step 856978 / 956000 | TPS 17.9299 | ETA 01:32:02
Time 03:10:07 | Step 857154 / 956000 | TPS 17.4253 | ETA 01:34:32
Time 03:10:17 | Step 857332 / 956000 | TPS 17.7967 | ETA 01:32:24
Time 03:10:27 | Step 857510 / 956000 | TPS 17.6329 | ETA 01:33:05
Time 03:10:37 | Step 857691 / 956000 | TPS 17.9707 | ETA 01:31:10
Time 03:10:47 | Step 857874 / 956000 | TPS 18.1995 | ETA 01:29:51
Time 03:10:57 | Step 858059 / 956000 | TPS 18.4875 | ETA 01:28:17
Time 03:11:07 | Step 858244 / 956000 | TPS 18.3778 | ETA 01:28:39
Time 03:11:17 | Step 858425 / 956000 | TPS 18.095 | ETA 01:29:52
Time 03:11:27 | Step 858609 / 956000 | TPS 18.3802 | ETA 0

In [ ]:
system.get_metadata()